In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
corpus='you say goodbye and i say hello .'

### 전처리

In [3]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

def conver_one_hot(corpus, window_size=1):

    le = LabelEncoder()
    enc=OneHotEncoder()
    
    corpus_list=corpus.split()
    
    le_list=le.fit_transform(corpus_list).reshape(len(corpus_list),-1)
    enc.fit(le_list)
    
    corpus_oh=enc.transform(le_list).toarray().tolist()
    
    contexts = []
    target = []
    for i, co in enumerate(corpus_oh):
        if (i < window_size) or (i >= (len(corpus_oh)-window_size)):
            continue

        contexts.append([corpus_oh[i-window_size],corpus_oh[i+window_size]])
        target.append(co)
        
    contexts = np.array(contexts)
    target = np.array(target)
    return contexts, target

In [4]:
contexts, target = conver_one_hot(corpus, 1)

/usr/local/Caskroom/miniconda/base/envs/artificial/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [7]:
contexts

array([[[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0.]]])

## 모델 만들기

In [12]:
# pre, pro로 나누기위해 shape 변환
contexts=tf.transpose(a=contexts, perm=[1,0,2]) # perm index를 바꿈

In [14]:
contexts, target

(<tf.Tensor: id=2, shape=(2, 6, 7), dtype=float64, numpy=
 array([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.]],
 
        [[0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0.]]])>, array([[0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0.]]))

In [16]:
target.shape, contexts.shape 

((6, 7), TensorShape([2, 6, 7]))

In [17]:
input_shape = (7,7)

In [18]:
from tensorflow.keras.layers import Input, Dense, Embedding, Dot, Reshape
from tensorflow.keras.models import Model

def Word2Vec_classification(input_shape):
    inputs1 = Input(shape=input_shape[0]) # input 2개중에 1개가 행렬 1x7로 이루어져있기 때문에 7이 들어감
    inputs2 = Input(shape=input_shape[1]) # 두번째 입력 값
    x1 = Dense(3, name='pre')(inputs1)  # matmul 부분 Dense는 처음 weight rand까지 다
    x2 = Dense(3, name='post')(inputs2)

    dot = tf.add(x1,x2) * 0.5 # 책에있는 공식이다 
        
    out = Dense(7, activation='softmax', name='output')(dot) #Weight x dot을 한후 소프트 맥스까지 한다.
    
    skipgram = Model(inputs=[inputs1, inputs2], outputs=out)
    
    return skipgram

CBOW = Word2Vec_classification(input_shape)
CBOW.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
pre (Dense)                     (None, 3)            24          input_1[0][0]                    
__________________________________________________________________________________________________
post (Dense)                    (None, 3)            24          input_2[0][0]                    
______________________________________________________________________________________________

In [20]:
CBOW.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])#adam = 통용적으로쓴다 metrics categorical_crossentropy 다수 binary_엔트로피는 2개할때 

In [21]:
CBOW.fit(
    [contexts[0],contexts[1]], #입력
    target, # 정답
    batch_size=3,# input 3덩어리 씩 입력하여 진행 하겠다.
    epochs=1000 # 몇바퀴
#     verbose=1
)

Train on 6 samples
Epoch 1/1000
6/6 [==============================] - 1s 143ms/sample - loss: 1.8107 - accuracy: 0.1667
Epoch 2/1000
6/6 [==============================] - 0s 917us/sample - loss: 1.8061 - accuracy: 0.1667
Epoch 3/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.8022 - accuracy: 0.1667
Epoch 4/1000
6/6 [==============================] - 0s 731us/sample - loss: 1.7978 - accuracy: 0.1667
Epoch 5/1000
6/6 [==============================] - 0s 901us/sample - loss: 1.7938 - accuracy: 0.1667
Epoch 6/1000
6/6 [==============================] - 0s 837us/sample - loss: 1.7901 - accuracy: 0.1667
Epoch 7/1000
6/6 [==============================] - 0s 775us/sample - loss: 1.7859 - accuracy: 0.1667
Epoch 8/1000
6/6 [==============================] - 0s 749us/sample - loss: 1.7820 - accuracy: 0.1667
Epoch 9/1000
6/6 [==============================] - 0s 885us/sample - loss: 1.7778 - accuracy: 0.1667
Epoch 10/1000
6/6 [==============================] - 0s 878us/sam

6/6 [==============================] - 0s 912us/sample - loss: 1.2396 - accuracy: 1.0000
Epoch 161/1000
6/6 [==============================] - 0s 969us/sample - loss: 1.2362 - accuracy: 1.0000
Epoch 162/1000
6/6 [==============================] - 0s 846us/sample - loss: 1.2328 - accuracy: 1.0000
Epoch 163/1000
6/6 [==============================] - 0s 931us/sample - loss: 1.2295 - accuracy: 1.0000
Epoch 164/1000
6/6 [==============================] - 0s 966us/sample - loss: 1.2259 - accuracy: 1.0000
Epoch 165/1000
6/6 [==============================] - 0s 981us/sample - loss: 1.2226 - accuracy: 1.0000
Epoch 166/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.2192 - accuracy: 1.0000
Epoch 167/1000
6/6 [==============================] - 0s 914us/sample - loss: 1.2161 - accuracy: 1.0000
Epoch 168/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.2124 - accuracy: 1.0000
Epoch 169/1000
6/6 [==============================] - 0s 853us/sample - loss: 1.209

6/6 [==============================] - 0s 1ms/sample - loss: 0.7587 - accuracy: 1.0000
Epoch 319/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7561 - accuracy: 1.0000
Epoch 320/1000
6/6 [==============================] - 0s 932us/sample - loss: 0.7535 - accuracy: 1.0000
Epoch 321/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7509 - accuracy: 1.0000
Epoch 322/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7484 - accuracy: 1.0000
Epoch 323/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7459 - accuracy: 1.0000
Epoch 324/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7433 - accuracy: 1.0000
Epoch 325/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7407 - accuracy: 1.0000
Epoch 326/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7382 - accuracy: 1.0000
Epoch 327/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.7356 - accuracy: 

6/6 [==============================] - 0s 1ms/sample - loss: 0.4286 - accuracy: 1.0000
Epoch 477/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4270 - accuracy: 1.0000
Epoch 478/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4254 - accuracy: 1.0000
Epoch 479/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4236 - accuracy: 1.0000
Epoch 480/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4221 - accuracy: 1.0000
Epoch 481/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4204 - accuracy: 1.0000
Epoch 482/1000
6/6 [==============================] - 0s 942us/sample - loss: 0.4190 - accuracy: 1.0000
Epoch 483/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.4174 - accuracy: 1.0000
Epoch 484/1000
6/6 [==============================] - 0s 937us/sample - loss: 0.4156 - accuracy: 1.0000
Epoch 485/1000
6/6 [==============================] - 0s 963us/sample - loss: 0.4142 - accura

6/6 [==============================] - 0s 1ms/sample - loss: 0.2299 - accuracy: 1.0000
Epoch 636/1000
6/6 [==============================] - 0s 947us/sample - loss: 0.2292 - accuracy: 1.0000
Epoch 637/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2282 - accuracy: 1.0000
Epoch 638/1000
6/6 [==============================] - 0s 985us/sample - loss: 0.2273 - accuracy: 1.0000
Epoch 639/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2263 - accuracy: 1.0000
Epoch 640/1000
6/6 [==============================] - 0s 961us/sample - loss: 0.2254 - accuracy: 1.0000
Epoch 641/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2246 - accuracy: 1.0000
Epoch 642/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2237 - accuracy: 1.0000
Epoch 643/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2229 - accuracy: 1.0000
Epoch 644/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2220 - accura

Epoch 793/1000
6/6 [==============================] - 0s 976us/sample - loss: 0.1257 - accuracy: 1.0000
Epoch 794/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1253 - accuracy: 1.0000
Epoch 795/1000
6/6 [==============================] - 0s 890us/sample - loss: 0.1248 - accuracy: 1.0000
Epoch 796/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1244 - accuracy: 1.0000
Epoch 797/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1239 - accuracy: 1.0000
Epoch 798/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1235 - accuracy: 1.0000
Epoch 799/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1230 - accuracy: 1.0000
Epoch 800/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1226 - accuracy: 1.0000
Epoch 801/1000
6/6 [==============================] - 0s 996us/sample - loss: 0.1221 - accuracy: 1.0000
Epoch 802/1000
6/6 [==============================] - 0s 960us/sample - loss

6/6 [==============================] - 0s 1ms/sample - loss: 0.0737 - accuracy: 1.0000
Epoch 952/1000
6/6 [==============================] - 0s 933us/sample - loss: 0.0735 - accuracy: 1.0000
Epoch 953/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0733 - accuracy: 1.0000
Epoch 954/1000
6/6 [==============================] - 0s 991us/sample - loss: 0.0730 - accuracy: 1.0000
Epoch 955/1000
6/6 [==============================] - 0s 987us/sample - loss: 0.0728 - accuracy: 1.0000
Epoch 956/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0725 - accuracy: 1.0000
Epoch 957/1000
6/6 [==============================] - 0s 932us/sample - loss: 0.0723 - accuracy: 1.0000
Epoch 958/1000
6/6 [==============================] - 0s 968us/sample - loss: 0.0721 - accuracy: 1.0000
Epoch 959/1000
6/6 [==============================] - 0s 951us/sample - loss: 0.0719 - accuracy: 1.0000
Epoch 960/1000
6/6 [==============================] - 0s 983us/sample - loss: 0.0717 

### 분산표현

In [22]:
np.save('CBOW_emb.npy', CBOW.get_weights()[0])

In [23]:
W_in = np.load(open('CBOW_emb.npy', 'rb'))

In [24]:
W_in

array([[-0.7727148 , -0.6154561 , -0.20414388],
       [-0.17119811, -1.4189329 ,  1.7449336 ],
       [-1.1517683 , -1.010803  , -0.6347144 ],
       [ 0.26659048, -0.21212399,  0.08505577],
       [-1.2657375 ,  1.6528533 , -1.0529145 ],
       [ 1.922609  , -0.37548652, -1.2594767 ],
       [-0.6537116 ,  0.46353006, -0.7326176 ]], dtype=float32)

### 모델 저장

In [25]:
model_json = CBOW.to_json()
with open("CBOW_emb_scratch.json", "w") as json_file : 
    json_file.write(model_json)
    
CBOW.save_weights("CBOW_emb_scratch.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
from tensorflow.keras.models import model_from_json 
with open("CBOW_emb_scratch.json", "r") as json_file : 
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("CBOW_emb_scratch.h5")
print("Loaded model from disk")

Loaded model from disk


In [27]:
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### train, test data 나누지 않음, data가 너무 적어서 모델을 만드는데 의의를 둠

In [28]:
target_pre=loaded_model.predict([contexts[0],contexts[1]])# 1000번 train된 weight으로 테스데이터를 넣어서 진행해야하지 만 귀찮아서 무리가 가지고 있는것을 넣음

In [29]:
target_pre

array([[3.16421571e-03, 1.45199709e-02, 6.68991450e-03, 5.14584733e-03,
        1.04119512e-03, 9.66514349e-01, 2.92456034e-03],
       [1.46087492e-03, 3.04541923e-02, 9.30486262e-01, 2.42556389e-02,
        1.37060590e-03, 9.51985735e-03, 2.45247595e-03],
       [7.17447605e-04, 9.08089161e-01, 2.83671729e-02, 1.79843119e-04,
        3.12965959e-02, 2.98072826e-02, 1.54257240e-03],
       [6.46061031e-03, 4.51862402e-02, 1.60122744e-03, 7.29223061e-03,
        9.31162357e-01, 8.31819721e-04, 7.46552041e-03],
       [1.77663437e-03, 1.12261865e-02, 7.23191956e-03, 3.03073181e-03,
        4.03117912e-04, 9.74610150e-01, 1.72124151e-03],
       [1.74723417e-02, 1.10530958e-03, 3.14045586e-02, 9.23032224e-01,
        8.83055199e-03, 5.32811647e-03, 1.28268180e-02]], dtype=float32)

In [23]:
def accuracy_euni(y_target, y_pred):
    y_true, pred = y_target.argmax(axis=1), y_pred.argmax(axis=1)
    
    return np.mean(pred == y_true)

In [24]:
accuracy_euni(target,target_pre)

1.0

In [22]:
target_pre.argmax(axis=1) # argmax?? 값이 max axis(1행 0열) 

array([5, 2, 1, 4, 5, 3])

In [18]:
target

array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])